# Loan API Endpoint

In [2]:
import requests as r
from bs4 import BeautifulSoup as bs
import json
from pprint import pp
import pandas as pd

url = "https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json"
response = r.get(url)
soup = bs(page.content)
pp(response)

<Response [200]>


In [4]:
data = response.json()
pp(data)


[{'Application_ID': 'LP001002',
  'Gender': 'Male',
  'Married': 'No',
  'Dependents': '0',
  'Education': 'Graduate',
  'Self_Employed': 'No',
  'Credit_History': 1,
  'Property_Area': 'Urban',
  'Income': 'medium',
  'Application_Status': 'Y'},
 {'Application_ID': 'LP001003',
  'Gender': 'Male',
  'Married': 'Yes',
  'Dependents': '1',
  'Education': 'Graduate',
  'Self_Employed': 'No',
  'Credit_History': 1,
  'Property_Area': 'Rural',
  'Income': 'medium',
  'Application_Status': 'N'},
 {'Application_ID': 'LP001005',
  'Gender': 'Male',
  'Married': 'Yes',
  'Dependents': '0',
  'Education': 'Graduate',
  'Self_Employed': 'Yes',
  'Credit_History': 1,
  'Property_Area': 'Urban',
  'Income': 'low',
  'Application_Status': 'Y'},
 {'Application_ID': 'LP001006',
  'Gender': 'Male',
  'Married': 'Yes',
  'Dependents': '0',
  'Education': 'Not Graduate',
  'Self_Employed': 'No',
  'Credit_History': 1,
  'Property_Area': 'Urban',
  'Income': 'low',
  'Application_Status': 'Y'},
 {'Applica

In [5]:
df_api_loan= pd.DataFrame(data)

In [6]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import*

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

In [8]:
spark = SparkSession\
    .builder\
    .appName("loan_api")\
    .config("spark.jars","/Users/roy/Downloads/mysql-connector-j-8.0.32/mysql-connector-j-8.0.32.jar")\
    .getOrCreate()

23/02/28 11:41:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [10]:
df_spark_api = spark.createDataFrame(df_api_loan) 
df_spark_api.printSchema()
df_spark_api.show(1)


root
 |-- Application_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- Credit_History: long (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- Application_Status: string (nullable = true)

+--------------+------+-------+----------+---------+-------------+--------------+-------------+------+------------------+
|Application_ID|Gender|Married|Dependents|Education|Self_Employed|Credit_History|Property_Area|Income|Application_Status|
+--------------+------+-------+----------+---------+-------------+--------------+-------------+------+------------------+
|      LP001002|  Male|     No|         0| Graduate|           No|             1|        Urban|medium|                 Y|
+--------------+------+-------+----------+---------+-------------+------------

In [12]:
df_spark_api.write.format("jdbc") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "creditcard_capstone.loan_api") \
          .option("user", "root") \
            .option("password", "password") \
            .mode("overwrite")\
            .save()
df_spark_api.show(1)


+--------------+------+-------+----------+---------+-------------+--------------+-------------+------+------------------+
|Application_ID|Gender|Married|Dependents|Education|Self_Employed|Credit_History|Property_Area|Income|Application_Status|
+--------------+------+-------+----------+---------+-------------+--------------+-------------+------+------------------+
|      LP001002|  Male|     No|         0| Graduate|           No|             1|        Urban|medium|                 Y|
+--------------+------+-------+----------+---------+-------------+--------------+-------------+------+------------------+
only showing top 1 row

